In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
#https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who/data
df = pd.read_csv('data.csv')
df.head(3)

In [ ]:
import math
df_rel = df[['Life expectancy ', 'GDP']]
data = df_rel.to_dict()
l = len(list(data['GDP'].values()))
for k in data:
    data[k] = list(data[k].values())
final_data = [(data['GDP'][i],data['Life expectancy '][i]) for i in range(l)]
data = final_data
data = [(x,y) for x,y in data if not math.isnan(x) and not math.isnan(y)]
print(len(data))
data[:4]

First, we'll do some basic statistics on this dataset to get an understanding for what we're working with.

In [ ]:
gdps = [x[0] for x in data]
les = [x[1] for x in data]
min_gdp = min(gdps)
max_gdp = max(gdps)
min_le = min(les)
max_le = max(les)
avg_gdp = sum(gdps)/len(gdps)
avg_le = sum(les)/len(les)
print(f'Basic Statistics\n- [GDP] min: {min_gdp}; max: {max_gdp}; avg: {avg_gdp}')
print(f'- [LE] min: {min_le}; max: {max_le}; avg: {avg_le}')

In [ ]:
from plot import Plot
p = Plot()
p.plotGeneric(data=([gdps[i] for i in range(len(gdps)) if les[i] > avg_le and gdps[i] > avg_gdp],
                    [les[i] for i in range(len(gdps)) if les[i] > avg_le and gdps[i] > avg_gdp]),label='above both averages',
                    scatterArgs={'s':1.5,'color':'g'},
                    wait=True)
p._legend()
p.plotGeneric(data=([gdps[i] for i in range(len(gdps)) if les[i] > avg_le and gdps[i] <= avg_gdp],
                    [les[i] for i in range(len(gdps)) if les[i] > avg_le and gdps[i] <= avg_gdp]),label='above average LE',
                    scatterArgs={'s':1.5,'color':'y'},
                    wait=True)
p._legend()
p.plotGeneric(data=([gdps[i] for i in range(len(gdps)) if les[i] <= avg_le and gdps[i] > avg_gdp],
                    [les[i] for i in range(len(gdps)) if les[i] <= avg_le and gdps[i] > avg_gdp]),label='above average gdp',
                    scatterArgs={'s':1.5,'color':'b'},
                    wait=True)
p._legend()
p.plotGeneric(data=([gdps[i] for i in range(len(gdps)) if les[i] <= avg_le and gdps[i] <= avg_gdp],
                    [les[i] for i in range(len(gdps)) if les[i] <= avg_le and gdps[i] <= avg_gdp]),label='below both averages',
                    scatterArgs={'s':0.25,'color':'r'},
                    wait=True,
                    chart={'title':'life expectancy (country) vs. GDP, 2000-2015',
                           'xlabel':'GDP ($)',
                           'ylabel':'Life Expectancy (years)'})
plt.plot([1000,70000],[avg_le,avg_le],label='average LE',color='black')
plt.plot([avg_gdp,avg_gdp],[55,84.5],label='average GDP',color='teal')
plt.plot([])
plt.legend()
p._showPlt()

In [ ]:
cat1 = [i for i in range(len(gdps)) if gdps[i] < avg_gdp and les[i] < avg_le]
cat2 = [i for i in range(len(gdps)) if gdps[i] < avg_gdp and les[i] >= avg_le]
cat3 = [i for i in range(len(gdps)) if gdps[i] >= avg_gdp and les[i] < avg_le]
cat4 = [i for i in range(len(gdps)) if gdps[i] >= avg_gdp and les[i] >= avg_le]

def _extract(idxs,arr):
       out = []
       for i in idxs:
              out.append(arr[i])
       return out

def extract(idxs,*arrs):
       out = []
       for _,a in enumerate(*arrs):
              out.append(_extract(idxs,a))
       return tuple(out)

def _extract_category(predictions,cat_num):
       print(predictions)
       return [i for i,x in enumerate(predictions) if x == cat_num]

def extract_categories(predictions):
       out = []
       for cat_num in range(1,5):
              out.append(_extract_category(predictions,cat_num))
       for i in range(len(out)):
              print(len(out[i]),len(gdps),len(les))
              out[i] = extract(out[i],[gdps,les])
       return out

from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
import numpy as np
n = KNeighborsClassifier(n_neighbors=2)

categories = [1,2,3,4]
y = []
cats = [cat1,cat2,cat3,cat4]
for i,cat in enumerate(cats):
    for c in cat:
        y.append(i+1)
y = np.array(y)
X = np.array(list(zip(gdps,les)))
classifier = n.fit(X,y)
predictions = classifier.predict(X)
cats = extract_categories(predictions)
print(f'cats {cats}')
colors = ['g','y','b','r']
p.plotGeneric(data=cats[3],label='best',
                    scatterArgs={'s':1.5,'color':'g'},
                    wait=True)
p._legend()
p.plotGeneric(data=cats[1],label='bad gdp, good le',
                    scatterArgs={'s':1.5,'color':'y'},
                    wait=True)
p._legend()
p.plotGeneric(data=cats[2],label='good gdp, bad le',
                    scatterArgs={'s':1.5,'color':'b'},
                    wait=True)
p._legend()
p.plotGeneric(data=cats[0],label='worst',
                    scatterArgs={'s':0.2,'color':'r'},
                    wait=True,
                    chart={'title':'life expectancy (country) vs. GDP, 2000-2015 [KNN]',
                           'xlabel':'GDP ($)',
                           'ylabel':'Life Expectancy (years)'})
plt.plot([1000,70000],[avg_le,avg_le],label='average LE',color='black')
plt.plot([avg_gdp,avg_gdp],[55,84.5],label='average GDP',color='teal')
plt.legend()
p._showPlt()
print('best',len(cats[3][0]))
print('good le',len(cats[1][0]))
print('good gdp',len(cats[2][0]))
print('worst',len(cats[0][0]))
print("Test set score: {:.2f}".format(np.mean(predictions == y)))


In [ ]:
len(cats),len(cats[0]),len(cats[0][0])

In [ ]:
!pip3 install mglearn --q

In [ ]:
from matplotlib import pyplot as plt
import mglearn
fig, axes = plt.subplots(1, 3, figsize=(10, 3))
for n_neighbors, ax in zip([1, 6, 40], axes):
# the fit method returns the object self, so we can instantiate
# and fit in one line
    clf = KNeighborsClassifier(n_neighbors=n_neighbors).fit(X, y) 
    mglearn.plots.plot_2d_separator(clf, X, fill=True, eps=0.05, ax=ax, alpha=.4) 
    mglearn.discrete_scatter(X[:, 0], X[:, 1], y, ax=ax,s=5)
    ax.set_title("{} neighbor(s)".format(n_neighbors))
    ax.set_xlabel("feature 0",fontsize=10)
    ax.set_ylabel("feature 1",fontsize=2)
_ = axes[0].legend(loc=3)
# Intro to ML w py. pg 36

In [ ]:
rn = RadiusNeighborsClassifier(radius=1.5)
classifier = rn.fit(X,y)
predictions = classifier.predict(X)
cats = extract_categories(predictions)
p.plotGeneric(data=cats[3],label='best',
                    scatterArgs={'s':1.5,'color':'g'},
                    wait=True)
p._legend()
p.plotGeneric(data=cats[1],label='bad gdp, good le',
                    scatterArgs={'s':1.5,'color':'y'},
                    wait=True)
p._legend()
p.plotGeneric(data=cats[2],label='good gdp, bad le',
                    scatterArgs={'s':1.5,'color':'b'},
                    wait=True)
p._legend()
p.plotGeneric(data=cats[0],label='worst',
                    scatterArgs={'s':1.5,'color':'r'},
                    wait=True,
                    chart={'title':'life expectancy (country) vs. GDP, 2000-2015 [1-R_NN]',
                           'xlabel':'GDP ($)',
                           'ylabel':'Life Expectancy (years)'})
plt.plot([1000,70000],[avg_le,avg_le],label='average LE',color='black')
plt.plot([avg_gdp,avg_gdp],[55,84.5],label='average GDP',color='teal')
plt.legend()
p._showPlt()
print('best',len(cats[3][0]))
print('good le',len(cats[1][0]))
print('good gdp',len(cats[2][0]))
print('worst',len(cats[0][0]))

In [ ]:
#todo: knn, other clustering

In [1]:
from plot import Plot
p = Plot()

# cat1 = [i for i in range(len(gdps)) if gdps[i] < avg_gdp and les[i] < avg_le]
# cat2 = [i for i in range(len(gdps)) if gdps[i] < avg_gdp and les[i] >= avg_le]
# cat3 = [i for i in range(len(gdps)) if gdps[i] >= avg_gdp and les[i] < avg_le]
# cat4 = [i for i in range(len(gdps)) if gdps[i] >= avg_gdp and les[i] >= avg_le]
m4 = lambda t: t[0] < p.avgs[0] and t[1] < p.avgs[1]
m3 = lambda t: t[0] < p.avgs[0] and t[1] >= p.avgs[1]
m2 = lambda t: t[0] >= p.avgs[0] and t[1] < p.avgs[1]
m1 = lambda t: t[0] >= p.avgs[0] and t[1] >= p.avgs[1]
members = [m1,m2,m3,m4]
cat_info = {'best':m1,'good gdp':m2,'good le':m3,'worst':m4}
p.clustering('GDP','Life expectancy ',cat_info=cat_info)
#     def clustering(self,x_name,y_name,cat_info,plot=1):


       Country  Year      Status  Life expectancy   Adult Mortality  \
0  Afghanistan  2015  Developing              65.0            263.0   
1  Afghanistan  2014  Developing              59.9            271.0   
2  Afghanistan  2013  Developing              59.9            268.0   
3  Afghanistan  2012  Developing              59.5            272.0   
4  Afghanistan  2011  Developing              59.2            275.0   

   infant deaths  Alcohol  percentage expenditure  Hepatitis B  Measles   ...  \
0             62     0.01               71.279624         65.0      1154  ...   
1             64     0.01               73.523582         62.0       492  ...   
2             66     0.01               73.219243         64.0       430  ...   
3             69     0.01               78.184215         67.0      2787  ...   
4             71     0.01                7.097109         68.0      3013  ...   

   Polio  Total expenditure  Diphtheria    HIV/AIDS         GDP  Population  \
0    6.

IndexError: tuple index out of range

In [ ]:
cat_info = {'best': lambda t: t }
p.clustering('GDP','Life expectancy ',cat_info={'z':lambda t: , 'LE'})
